In [1]:
import os


In [2]:
%pwd

'f:\\end-to-end-deep_learning-project\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\end-to-end-deep_learning-project'

# In the model_trainer_stage, your goal is simple:

⭐ Take the updated base model → Train it → Save the trained model.

No confusion.
No extras.
Here are the only tasks the training stage must do.

✅ 1. Load the Updated Base Model

You previously saved:

base_model.pt → frozen base

base_model_updated.pt → base + new classifier

Training stage ALWAYS loads:

updated_base_model_path     (e.g. base_model_updated.pth)

✅ 2. Load Training Data

Your config already gives you:

training_data = Path(training_data_dir)


Model trainer must:

Use ImageFolder

Apply transforms

Create DataLoader

✅ 3. Set Up Loss & Optimizer

Because you already

froze feature layers

unfroze classifier only

You must define:

CrossEntropyLoss

Adam(model.classifier.parameters(), lr=...)

✅ 4. Run the Training Loop

For each epoch:

iterate through dataloader

forward pass

compute loss

backward pass

optimizer step

accumulate loss

✅ 5. Save the Trained Model

Save as:

trained_model_path (e.g. artifacts/training/model.pth)


This is your final trained model.

# 🌟 What your trainer stage code will logically look like

(no code, just logic)

Load updated_base_model

Load training dataset from folder

Apply transforms (resize, normalize)

Create DataLoader

Move model to device

Define loss + optimizer

For each epoch:

    For each batch:

        Forward pass

        Compute loss

        Backward pass
        
        Update parameters

    Print epoch loss
    
Save trained model


In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [4]:
import os
#import urllib.request as request
#from zipfile import ZipFile
import time